# Expo Ingenierias (español)

### Librerías y Base de Datos

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
import itertools

# Importar las bases de datos necesarias para el ejemplo
sat = pd.read_csv('sat.csv')
tiempo = pd.read_csv('tiempo.csv')
costo = pd.read_csv('costo.csv')

# Convertir en string los indices de neustras bases de datos para ser manejadas de mejor manera
sat.index = sat.index.astype('str')
tiempo.index = tiempo.index.astype('str')
costo.index = costo.index.astype('str')

In [ ]:
# Crear el diccionario con los lugares según cada nodo
diccionario = {
    '0' : ['Hotel Casa Reyna'],
    '1' : ['Catedral Basílica de Puebla'],
    '2' : ['Capilla del Rosario'],
    '3' : ['Mercado el Parián'],
    '4' : ['Barrio del Artista'],
    '5' : ['Zócalo Puebla'],
    '6' : ['Callejón de los Sapos'],
    '7' : ['Iglesia de Nuestra Señora de los Remedios'],
    '8' : ['Zona Arqueológica de Cholula'],
    '9' : ['Templo de San Francisco Acatepec'],
    '10' : ['Museo Fuertes de Loreto'],
    '11' : ['Africam Safari'],
    '12' : ['Calle de los Dulces'],
    '13' : ['Casa del Alfeñique'],
    '14' : ['Biblioteca Palafoxiana'],
    '15' : ['Museo Amparo'],
    '16' : ['Museo Internacional del Barroco'],
    '17' : ['Pasaje Histórico del 5 de Mayo'],
    '18' : ['Museo Regional de Cholula'],
    '19' : ['Volcán Cuexcomate'],
    '20' : ['Museo Regional De La Revolución Mexicana Casa De Los Hermanos Serdán']
}

diccionario = pd.DataFrame(diccionario, index=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

### Modelo de TSP

In [ ]:
def tsp(nodos_por_dia, tiempo_data):
  # Creacion de matriz solo con los nodos de ese dia
  tiempo_por_dia = tiempo_data.loc[nodos_por_dia, nodos_por_dia]

  # Remove node 0 from nodos_por_dia
  nodos_sin_cero = [n for n in nodos_por_dia if n != '0']

  # Generar todos las las combinaciones entre los nodos
  all_permutations_sin_cero = itertools.permutations(nodos_sin_cero)

  # Generar las variables vacias que se usaran
  min_route = None
  min_tiempo = float('inf')

  # Checar cada permutacion
  for permutation_sin_cero in all_permutations_sin_cero:
    # Meter el nodo 0 (hotel) and principio y al final del ruteo
    permutation = ('0',) + permutation_sin_cero + ('0',)

    # Inicializar la variable
    current_tiempo = 0

    # Ver el tiempo de ese ruteo
    for i in range(len(permutation) - 1):
      current_tiempo += tiempo_por_dia.loc[permutation[i], permutation[i + 1]]

    # Ver si es el tiempo mas chico se que puede tener
    if current_tiempo < min_tiempo:
      min_tiempo = current_tiempo
      min_route = permutation

  return min_route, min_tiempo

### Maxima Satisfacción con TSP

In [ ]:
def Max_Satisfaccion(sat, tiempo, costo, diccionario, horas, presupuesto, dias):
    # Creacion de una copia de matriz de tiempo para el uso correcto del TSP
    tiempo_data = pd.DataFrame(tiempo, index=[str(i) for i in range(tiempo.shape[0])])

    # Inicializacion de valores iniciales
    sat_total = 0
    tiempo_total = 0
    costo_total = 0

    # Obtencion de la media de los regesos al hote, colchon de tiempos
    media_regreso_hotel = tiempo.iloc[1:, 0].mean()
    # Conversion de el tiempo de restrinccion diaria
    minutos = ((horas - 2)*60) -  media_regreso_hotel*2

    # Copia que se modificara de la matriz de satisfaccion
    modified_df = sat.copy()
    modified_df = pd.DataFrame(modified_df)

    # Hacer la iteracion la canitdad de dias que el usuario decidio
    for i in range(dias):
      # Inicializacion de valores diarias
      tiempo_dia = 0
      costo_dia = 0
      sat_dia = 0

      # Parar la creacion de rutas
      if (modified_df == 0).all().all():
        print('Ya no se pueden crear mas rutas.')
        break

      # Creacion del conjunto para gurdar los nodos visitados diarios
      nodos_por_dia = []

      print(f"--- COMIENZO DEL DÍA: {i + 1} --- \n")

      # Ciclo por dia para elgoritmo de greedy
      while tiempo_dia < minutos and not modified_df.empty:

            # Enontrar el valor mas alto en la matriz de satisfaccion
            overall_max_value = modified_df.values.max()

            # Buscar la columna y fila donde este valor maximo de sat
            # Si es primera iteracion inicializar el primer origen
            if tiempo_dia == 0:
                max_row, max_col = modified_df.stack().idxmax()
                nodos_por_dia.append(max_col)
                if modified_df.empty:
                  max_row = previous_max_col
                  max_col = modified_df.loc[max_row].idxmin()
                  nodos_por_dia.append(max_col)

            # Si ya es depsues de la primera iteracion buscar la satisfaccion alta del destino anterior
            else:
                max_row = previous_max_col
                max_col = modified_df.loc[max_row].idxmax()
                nodos_por_dia.append(max_col)
                if modified_df.empty:
                  max_row = previous_max_col
                  max_col = modified_df.loc[max_row].idxmin()
                  nodos_por_dia.append(max_col)

            # Al tiempo sumarle el tiempo de matriz tiempo, misma posicion de maxima satisfaccion
            tiempo_dia += tiempo.loc[max_row, max_col]

            # Al costo sumarle el costo de matriz costo, misma posicion de maxima satisfaccion
            costo_dia += costo.loc[max_row, max_col]

            # Matriz satisfaccion remplazar destino utilizado por ceros
            modified_df[max_col] = 0

            # Establecer que el destino anterioir es ahora nuestro origen siguiente
            previous_max_row = max_row
            previous_max_col = max_col

            # Quebrar el algoritmo si ya no hay mas nodos que visitar
            if (modified_df == 0).all().all():
                print('Fin del ruteo.')
                break

            # Quebrar si se llega al limite de tiempo como de presupuesto diario aproximado del dia
            if tiempo_dia >= minutos or costo_dia >= (presupuesto/dias):

              # Regreso al punto 0
              if previous_max_col != 0:
                  max_row = previous_max_col
                  max_col = modified_df.loc[max_row].idxmin()
                  nodos_por_dia.append(max_col)

                  # Llamar a la funcion de TSP con los nodos seleccioando y visitados en el dia
                  ruta_TSP, tiempo_TSP = tsp(nodos_por_dia, tiempo_data)

                  # Inicializacion de valores diarias
                  tiempo_dia_pre_tsp = tiempo_dia
                  tiempo_total_pre_tsp = tiempo_dia
                  tiempo_dia = 0
                  costo_dia = 0
                  sat_dia = 0

                  # Iteracion de muestra de la ruta optimizada por el TSP
                  for j in range(len(ruta_TSP) - 1):
                    origen = ruta_TSP[j]
                    destino = ruta_TSP[j + 1]

                    # Suma de valores seleccionados de los nodos seleccionados
                    tiempo_dia += tiempo.loc[origen, destino]
                    costo_dia += costo.loc[origen, destino]
                    sat_dia += sat.loc[origen, destino]

                    # Mostrar el tiempo de viaje con uso de los nombres de los nodos
                    print(f"{diccionario.loc[origen, origen]} --> {diccionario.loc[destino,destino]}")

                  # Sumar las variables a las sumas totales del viaje
                  costo_total += costo_dia
                  sat_total += sat_dia
                  tiempo_total += tiempo_dia

                  # Imprimir los valores importantes de nuestro dia
                  print("\n")
                  print(f"Tiempo acumulado por día antes de TSP: {tiempo_dia_pre_tsp/60} horas")
                  print(f"Tiempo acumulado por día despues de TSP: {tiempo_dia/60} horas")
                  print(f"Costo acumulado por día: ${costo_dia}")
                  print(f"Satisfacción acumulada del día: {sat_dia}")
                  # print(f"Tiempo optimizado por el TSP: {tiempo_dia-tiempo_TSP}")

              # Quebrar el dia e iterar el dia
              print(f"\n--- FIN DEL DÍA: {i + 1} ---")
              print("\n")
              i = i + 1
              break

    # Imprimir valores finales del viaje
    print("\n")
    print(f"Satisfaccion acumulada del viaje: ", sat_total)
    print(f"Costo acumulada del viaje: ${costo_total} , Presupuesto sobrante: ${presupuesto - costo_total}")
    #print(f"Tiempo optimizado: {(tiempo_total_pre_tsp - tiempo_total)}")



In [ ]:
# Ingresar datos iniciales
dias = 3
horas = 12
presupuesto = 5000


# Llamar a la función
Max_Satisfaccion(sat, tiempo, costo, diccionario, horas, presupuesto, dias)


--- COMIENZO DEL DÍA: 1 --- 

Hotel Casa Reyna --> Zócalo Puebla
Zócalo Puebla --> Zona Arqueológica de Cholula
Zona Arqueológica de Cholula --> Africam Safari
Africam Safari --> Hotel Casa Reyna


Tiempo acumulado por día antes de TSP: 10.566666666666666 horas
Tiempo acumulado por día despues de TSP: 10.916666666666666 horas
Costo acumulado por día: $940.0
Satisfacción acumulada del día: 26.0516

--- FIN DEL DÍA: 1 ---


--- COMIENZO DEL DÍA: 2 --- 

Hotel Casa Reyna --> Barrio del Artista
Barrio del Artista --> Mercado el Parián
Mercado el Parián --> Callejón de los Sapos
Callejón de los Sapos --> Museo Amparo
Museo Amparo --> Museo Internacional del Barroco
Museo Internacional del Barroco --> Capilla del Rosario
Capilla del Rosario --> Hotel Casa Reyna


Tiempo acumulado por día antes de TSP: 9.533333333333333 horas
Tiempo acumulado por día despues de TSP: 9.5 horas
Costo acumulado por día: $273.5
Satisfacción acumulada del día: 42.8108

--- FIN DEL DÍA: 2 ---


--- COMIENZO DEL DÍA

### Optimum Route

In [ ]:
def Optimum_Route():
  # Dar la bienvenida a los usuarios
  print(f"¡BIENVENIDO A OPTIMUM ROUTE!")
  print(f"Es un placer atenderte")

  nombre = input("¿Cuál es tu nombre? ")
  print('\n')
  print(f"Hola, {nombre}!")
  print('\n')

  # Pide al usuario que ingrese la duracion de su viaje
  while True:
    try:
      dias = int(input("¿Cuántos días quieres que dure el tour? "))
      print('\n')
      break
    except ValueError:
      print("Por favor, introduce un caracter numerico: ")
      print('\n')

  # Pide al usuario que ingrese las horas de turisteo diarias
  while True:
    try:
      horas = int(input("¿Cuántas horas al día te gustaría turistear? "))
      if 1 <= horas <= 24:
        print('\n')
        break
      else:
        print('Lo siento, introduce un número entre el 1 y el 24, por favor')
        print('\n')
    except ValueError:
      print("Lo siento, por favor introduce un caracter numerico: \n")


  # Pide al usuario que ingrese la duracion de su viaje
  # Pide el presupuesto del viaje
  while True:
    try:
      presupuesto = float(input("¿De cuánto es tu presupuesto para movilidad y de paga de lugares a conocer?"))
      print('\n')
      break
    except ValueError:
      print("Lo siento, por favor introduce un caracter numerico: \n")

  print(f'Esta va a ser tu ruta de viaje {nombre}')
  print('\n')


  # Ahora puedes llamar a la función con los valores adecuados
  Max_Satisfaccion(sat, tiempo, costo, diccionario, horas, presupuesto, dias)

  print('\n')
  print(f'¡Muchas gracias por elegirnos, Optimun Routes está feliz de tenerte!')

### Llamar Optimum Route

In [ ]:
Optimum_Route()


¡BIENVENIDO A OPTIMUM ROUTE!
Es un placer atenderte
¿Cuál es tu nombre? Montse


Hola, Montse!


¿Cuántos días quieres que dure el tour? 4


¿Cuántas horas al día te gustaría turistear? 5


¿De cuánto es tu presupuesto para movilidad y de paga de lugares a conocer?1000


Esta va a ser tu ruta de viaje Montse


--- COMIENZO DEL DÍA: 1 --- 

Hotel Casa Reyna --> Zócalo Puebla
Zócalo Puebla --> Africam Safari
Africam Safari --> Hotel Casa Reyna


Tiempo acumulado por día antes de TSP: 6.883333333333334 horas
Tiempo acumulado por día despues de TSP: 7.6 horas
Costo acumulado por día: $623.5
Satisfacción acumulada del día: 17.6814

--- FIN DEL DÍA: 1 ---


--- COMIENZO DEL DÍA: 2 --- 

Hotel Casa Reyna --> Zona Arqueológica de Cholula
Zona Arqueológica de Cholula --> Hotel Casa Reyna


Tiempo acumulado por día antes de TSP: 3.6 horas
Tiempo acumulado por día despues de TSP: 4.233333333333333 horas
Costo acumulado por día: $408.5
Satisfacción acumulada del día: 8.3702

--- FIN DEL DÍA: 2 ---

# Feria de Ciencias (ingles)

### Librerías y Base de Datos

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
import itertools

# Importar las bases de datos necesarias para el ejemplo
sat = pd.read_csv('sat.csv')
tiempo = pd.read_csv('tiempo.csv')
costo = pd.read_csv('costo.csv')

# Convertir en string los indices de neustras bases de datos para ser manejadas de mejor manera
sat.index = sat.index.astype('str')
tiempo.index = tiempo.index.astype('str')
costo.index = costo.index.astype('str')

In [ ]:
# Crear el diccionario con los lugares según cada nodo
diccionario = {
    '0' : ['Hotel Casa Reyna'],
    '1' : ['Catedral Basílica de Puebla'],
    '2' : ['Capilla del Rosario'],
    '3' : ['Mercado el Parián'],
    '4' : ['Barrio del Artista'],
    '5' : ['Zócalo Puebla'],
    '6' : ['Callejón de los Sapos'],
    '7' : ['Iglesia de Nuestra Señora de los Remedios'],
    '8' : ['Zona Arqueológica de Cholula'],
    '9' : ['Templo de San Francisco Acatepec'],
    '10' : ['Museo Fuertes de Loreto'],
    '11' : ['Africam Safari'],
    '12' : ['Calle de los Dulces'],
    '13' : ['Casa del Alfeñique'],
    '14' : ['Biblioteca Palafoxiana'],
    '15' : ['Museo Amparo'],
    '16' : ['Museo Internacional del Barroco'],
    '17' : ['Pasaje Histórico del 5 de Mayo'],
    '18' : ['Museo Regional de Cholula'],
    '19' : ['Volcán Cuexcomate'],
    '20' : ['Museo Regional De La Revolución Mexicana Casa De Los Hermanos Serdán']
}

diccionario = pd.DataFrame(diccionario, index=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

### Modelo de TSP

In [ ]:
def tsp(nodos_por_dia, tiempo_data):
  # Creacion de matriz solo con los nodos de ese dia
  tiempo_por_dia = tiempo_data.loc[nodos_por_dia, nodos_por_dia]

  # Remove node 0 from nodos_por_dia
  nodos_sin_cero = [n for n in nodos_por_dia if n != '0']

  # Generar todos las las combinaciones entre los nodos
  all_permutations_sin_cero = itertools.permutations(nodos_sin_cero)

  # Generar las variables vacias que se usaran
  min_route = None
  min_tiempo = float('inf')

  # Checar cada permutacion
  for permutation_sin_cero in all_permutations_sin_cero:
    # Meter el nodo 0 (hotel) and principio y al final del ruteo
    permutation = ('0',) + permutation_sin_cero + ('0',)

    # Inicializar la variable
    current_tiempo = 0

    # Ver el tiempo de ese ruteo
    for i in range(len(permutation) - 1):
      current_tiempo += tiempo_por_dia.loc[permutation[i], permutation[i + 1]]

    # Ver si es el tiempo mas chico se que puede tener
    if current_tiempo < min_tiempo:
      min_tiempo = current_tiempo
      min_route = permutation

  return min_route, min_tiempo

### Maxima Satisfacción con TSP

In [ ]:
def Max_Satisfaccion(sat, tiempo, costo, diccionario, horas, presupuesto, dias):
    # Creacion de una copia de matriz de tiempo para el uso correcto del TSP
    tiempo_data = pd.DataFrame(tiempo, index=[str(i) for i in range(tiempo.shape[0])])

    # Inicializacion de valores iniciales
    sat_total = 0
    tiempo_total = 0
    costo_total = 0

    # Obtencion de la media de los regesos al hote, colchon de tiempos
    media_regreso_hotel = tiempo.iloc[1:, 0].mean()
    # Conversion de el tiempo de restrinccion diaria
    minutos = ((horas - 2)*60) -  media_regreso_hotel*2

    # Copia que se modificara de la matriz de satisfaccion
    modified_df = sat.copy()
    modified_df = pd.DataFrame(modified_df)

    # Hacer la iteracion la canitdad de dias que el usuario decidio
    for i in range(dias):
      # Inicializacion de valores diarias
      tiempo_dia = 0
      costo_dia = 0
      sat_dia = 0

      # Parar la creacion de rutas
      if (modified_df == 0).all().all():
        print('No more posible roots')
        break

      # Creacion del conjunto para gurdar los nodos visitados diarios
      nodos_por_dia = []

      print(f"--- START OF DAY: {i + 1} --- \n")

      # Ciclo por dia para elgoritmo de greedy
      while tiempo_dia < minutos and not modified_df.empty:

            # Enontrar el valor mas alto en la matriz de satisfaccion
            overall_max_value = modified_df.values.max()

            # Buscar la columna y fila donde este valor maximo de sat
            # Si es primera iteracion inicializar el primer origen
            if tiempo_dia == 0:
                max_row, max_col = modified_df.stack().idxmax()
                nodos_por_dia.append(max_col)
                if modified_df.empty:
                  max_row = previous_max_col
                  max_col = modified_df.loc[max_row].idxmin()
                  nodos_por_dia.append(max_col)

            # Si ya es depsues de la primera iteracion buscar la satisfaccion alta del destino anterior
            else:
                max_row = previous_max_col
                max_col = modified_df.loc[max_row].idxmax()
                nodos_por_dia.append(max_col)
                if modified_df.empty:
                  max_row = previous_max_col
                  max_col = modified_df.loc[max_row].idxmin()
                  nodos_por_dia.append(max_col)

            # Al tiempo sumarle el tiempo de matriz tiempo, misma posicion de maxima satisfaccion
            tiempo_dia += tiempo.loc[max_row, max_col]

            # Al costo sumarle el costo de matriz costo, misma posicion de maxima satisfaccion
            costo_dia += costo.loc[max_row, max_col]

            # Matriz satisfaccion remplazar destino utilizado por ceros
            modified_df[max_col] = 0

            # Establecer que el destino anterioir es ahora nuestro origen siguiente
            previous_max_row = max_row
            previous_max_col = max_col

            # Quebrar el algoritmo si ya no hay mas nodos que visitar
            if (modified_df == 0).all().all():
                print('End of route.')
                break

            # Quebrar si se llega al limite de tiempo como de presupuesto diario aproximado del dia
            if tiempo_dia >= minutos or costo_dia >= (presupuesto/dias):

              # Regreso al punto 0
              if previous_max_col != 0:
                  max_row = previous_max_col
                  max_col = modified_df.loc[max_row].idxmin()
                  nodos_por_dia.append(max_col)

                  # Llamar a la funcion de TSP con los nodos seleccioando y visitados en el dia
                  ruta_TSP, tiempo_TSP = tsp(nodos_por_dia, tiempo_data)

                  # Inicializacion de valores diarias
                  tiempo_dia_pre_tsp = tiempo_dia
                  tiempo_total_pre_tsp = tiempo_dia
                  tiempo_dia = 0
                  costo_dia = 0
                  sat_dia = 0

                  # Iteracion de muestra de la ruta optimizada por el TSP
                  for j in range(len(ruta_TSP) - 1):
                    origen = ruta_TSP[j]
                    destino = ruta_TSP[j + 1]

                    # Suma de valores seleccionados de los nodos seleccionados
                    tiempo_dia += tiempo.loc[origen, destino]
                    costo_dia += costo.loc[origen, destino]
                    sat_dia += sat.loc[origen, destino]

                    # Mostrar el tiempo de viaje con uso de los nombres de los nodos
                    print(f"{diccionario.loc[origen, origen]} --> {diccionario.loc[destino,destino]}")

                  # Sumar las variables a las sumas totales del viaje
                  costo_total += costo_dia
                  sat_total += sat_dia
                  tiempo_total += tiempo_dia

                  # Imprimir los valores importantes de nuestro dia
                  print("\n")
                  print(f"Time per day after TSP: {tiempo_dia/60} hr")
                  print(f"Budget per day: ${costo_dia}")
                  print(f"Satisfaction per day: {sat_dia}")
                  # print(f"Tiempo optimizado por el TSP: {tiempo_dia-tiempo_TSP}")

              # Quebrar el dia e iterar el dia
              print(f"\n--- END OF DAY: {i + 1} ---")
              print("\n")
              i = i + 1
              break

    # Imprimir valores finales del viaje
    print("\n")
    print(f"Total satisfaction of the trip: ", sat_total)
    print(f"Total budget of the trip: ${costo_total} , surplus budget: ${presupuesto - costo_total}")
    #print(f"Tiempo optimizado: {(tiempo_total_pre_tsp - tiempo_total)}")



In [ ]:
# Ingresar datos iniciales
days = 3
hours = 12
budget = 5000


# Llamar a la función
Max_Satisfaccion(sat, tiempo, costo, diccionario, hours, budget, days)


--- START OF DAY: 1 --- 

Hotel Casa Reyna --> Zócalo Puebla
Zócalo Puebla --> Zona Arqueológica de Cholula
Zona Arqueológica de Cholula --> Africam Safari
Africam Safari --> Hotel Casa Reyna


Time per day after TSP: 10.916666666666666 hr
Budget per day: $940.0
Satisfaction per day: 26.0516

--- END OF DAY: 1 ---


--- START OF DAY: 2 --- 

Hotel Casa Reyna --> Barrio del Artista
Barrio del Artista --> Mercado el Parián
Mercado el Parián --> Callejón de los Sapos
Callejón de los Sapos --> Museo Amparo
Museo Amparo --> Museo Internacional del Barroco
Museo Internacional del Barroco --> Capilla del Rosario
Capilla del Rosario --> Hotel Casa Reyna


Time per day after TSP: 9.5 hr
Budget per day: $273.5
Satisfaction per day: 42.8108

--- END OF DAY: 2 ---


--- START OF DAY: 3 --- 

Hotel Casa Reyna --> Museo Fuertes de Loreto
Museo Fuertes de Loreto --> Calle de los Dulces
Calle de los Dulces --> Museo Regional De La Revolución Mexicana Casa De Los Hermanos Serdán
Museo Regional De La Re

### Optimum Route

In [ ]:
def Optimum_Route():
  # Dar la bienvenida a los usuarios
  print(f"¡WELCOME TO OPTIMUM TRAVEL ROUTE!")
  print(f"It's a pleasure to serve you.")

  nombre = input("What's your name? ")
  print('\n')
  print(f"Hi, {nombre}!")
  print('\n')

  # Pide al usuario que ingrese la duracion de su viaje
  while True:
    try:
      dias = int(input("How many days do you want the tour to last? "))
      print('\n')
      break
    except ValueError:
      print("Please enter a numeric character: ")
      print('\n')

  # Pide al usuario que ingrese las horas de turisteo diarias
  while True:
    try:
      horas = int(input("How many hours a day would you like to tourist? "))
      if 1 <= horas <= 24:
        print('\n')
        break
      else:
        print('Sorry, enter a number between 1 and 24 please ')
        print('\n')
    except ValueError:
      print("Sorry, please enter a numeric character: \n")


  # Pide al usuario que ingrese la duracion de su viaje
  # Pide el presupuesto del viaje
  while True:
    try:
      presupuesto = float(input("How much is your budget for mobility and payment for places to visit?"))
      print('\n')
      break
    except ValueError:
      print("Sorry, please enter a numeric character: \n")

  print(f'This is going to be your travel route {nombre}')
  print('\n')


  # Ahora puedes llamar a la función con los valores adecuados
  Max_Satisfaccion(sat, tiempo, costo, diccionario, horas, presupuesto, dias)

  print('\n')
  print(f'Thank you very much for choosing us, Optimun Routes is happy to help you!')

### Llamar Optimum Route

In [ ]:
Optimum_Route()


¡WELCOME TO OPTIMUM TRAVEL ROUTE!
It's a pleasure to serve you.
